# Dependences

In [2]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")

cu_force

# Model & Aggregate

In [6]:
@time Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 100
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input = "../../data/init/Sphere"
    )
)
dump(Model)

FusionAGG = nothing
# @time @start_agg FusionAGG = FusionAggregate(
@time FusionAGG = FusionAggregate(
    [
        AggType(
            "HEK_1", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        ),
        AggType(
            "HEK_2", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    Model
)

  0.000002 seconds (3 allocations: 112 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 10000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 50
    nₛₐᵥₑ: Int64 100
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  7.281229 seconds (13.11 M allocations: 1.133 GiB, 3.55% gc time, 4.77% compilation time)


Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.001)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}), AggType("HEK_2", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.001)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2"]), Float32[-16.77 -4.62 -13.88; -14.77 -4.62 -13.88; … ; 15.77 4.62 13.88; 17.77 4.62 13.88], AggGeometry(Float32[15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27  …  15.27, 15.27, 15.27, 15.

# kNN

In [12]:
println("--------------------------------- Running CUDA kNN ---------------------------------")
@time nearest_neighbors(FusionAGG)
display(FusionAGG.Simulation.Neighbor.idx_red)
println("-------------------------------- Running CUDA Force --------------------------------")
@time cu_force(FusionAGG,1.0)
display(FusionAGG.Simulation.Force.F)

--------------------------------- Running CUDA kNN ---------------------------------
Threads = (32, 32) | Blocks  = (157, 157)
Threads = 256 | Blocks  = 20
Threads = (32, 32) | Blocks  = (157, 157)
  0.000275 seconds (229 allocations: 9.875 KiB)


21×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
  1   1   2   4   1   1   2   3   4  …  4949  4950  4951  4957  4958  4959
  2   2   3   5   4   2   3   7   9     4957  4958  4959  4964  4965  4966
  5   3   7   9   5   5   6   8  10     4958  4959  4960  4965  4966  4967
  6   6   8  10   6   6   7  13  15     4964  4965  4966  4971  4972  4973
 35   7  37  40  10   7   8  14  48     4965  4966  4967  4972  4973  4974
 36  36  38  41  11  11  12  44  49  …  4966  4967  4968  4973  4974  4975
 37  37  39  42  41  12  13  45  50     4997  4998  4999  5002  5003  5004
 42  38  44  49  42  42  43  46  57     4998  4999  5000  5003  5004  5005
 43  43  45  50  43  43  44  53  58     5002  5003  5004  5006  5006  5007
 51  44  53  58  50  44  45  54  66     5003  5004  5005  5007  5007  5008
  0  52   0   0  51  51  52  62   0  …  5004  5005  5008     0  5008     0
  0   0   0   0  59  52  53   0   0     5006  5007     0     0     0     0
  0   0   0   0   0  60  61   0   0     5007  5008

-------------------------------- Running CUDA Force --------------------------------
  0.000084 seconds (48 allocations: 3.250 KiB)


5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.000708019   0.000392189  -0.000212381
  0.0          -1.49224f-5    0.000364654
  0.000500444  -0.000880691   0.000364654
  0.0           0.000133741   0.000364654
 -0.000509339  -0.000141072  -0.00045103
  0.001         0.000164602   0.000364654
 -0.000708019   0.000571713  -0.000212382
 -0.000489364  -0.000717842   0.000364654
  0.000731929   0.000325632  -0.000211375
 -0.000500444  -0.00102452    0.000364828
 -0.000500444  -0.00102452    0.000364828
  0.000498284   0.000708258   0.000364828
  0.000706996   0.000251302  -0.000211375
  ⋮                          
  0.000498284  -0.000708258  -0.000364828
  0.000498284  -0.000708258  -0.000364828
  0.0           0.000158756  -0.000364828
  0.000464433  -0.000767159  -0.000364828
  0.000498284   0.000733273  -0.000364654
  0.000991081  -0.000149172  -0.000364654
 -0.000498284   0.000702412  -0.000364654
 -0.001        -0.000164602  -0.000364654
  8.91939f-6   -0.000725658   0.000452